In [ ]:
import nltk

nltk.download('stopwords')
!python -m spacy download pt_core_news_sm

In [ ]:
import os
from dotenv import load_dotenv

# Carregar variáveis do arquivo .env
load_dotenv()

server = "127.0.0.1"
port = "3306"
database = "glpi"
# Acessar as variáveis de ambiente
user = os.getenv('MARIADB_USER')
password = os.getenv('MARIADB_PASSWORD')

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Criar conexão com o MariaDB
engine = create_engine(f"mariadb+mariadbconnector://{user}:{password}@{server}:{port}/{database}")

# Ler os dados do banco de dados
sql = """
SELECT
	tickets.id              AS id
,	tickets.name            AS name
,	tickets.content         AS content
,	category.name           AS category
, 	category.completename   AS fullcategory
FROM glpi_tickets tickets
LEFT JOIN glpi_itilcategories category
ON tickets.itilcategories_id = category.id
"""
data = pd.read_sql(sql, engine)
data.info()

In [ ]:
def remove_noise(text):
    import re

    # Converte para minúsculas
    text = text.lower()

    # Remove pontuação
    # []: colchetes são usados para definir uma classe de caracteres.
    # ^: quando usado no início de uma classe de caracteres, o ^ nega a classe, ou seja, seleciona tudo que não está na classe.
    # \w: corresponde a qualquer caractere alfanumérico (letras e números, incluindo o caractere de sublinhado _)
    # \s: corresponde a qualquer espaço em branco (espaços, tabulações, quebras de linha).
    text = re.sub(r'[^\w\s]', '', text)

    return text

In [ ]:
def remove_stopwords(text):
    from nltk.corpus import stopwords
    # Obtém a lista de stopwords em português usando o NLTK e as converte para um conjunto para melhorar a eficiência da busca
    stop_words = set(stopwords.words('portuguese'))

    # Divide o texto em palavras, remove as stopwords e então junta as palavras restantes de volta em uma string
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

In [ ]:
data['clean_content'] = data['content'].apply(remove_noise)
data.head(5)

In [ ]:
data['clean_content'] = data['clean_content'].apply(remove_stopwords)
data.head(5)